# Pacific Atlantic Water Flow

https://leetcode.com/problems/pacific-atlantic-water-flow/

### Problem

There is an m x n rectangular island that borders both the Pacific Ocean and Atlantic Ocean. The Pacific Ocean touches the island's left and top edges, and the Atlantic Ocean touches the island's right and bottom edges.

The island is partitioned into a grid of square cells. You are given an m x n integer matrix heights where heights[r][c] represents the height above sea level of the cell at coordinate (r, c).

The island receives a lot of rain, and the rain water can flow to neighboring cells directly north, south, east, and west if the neighboring cell's height is less than or equal to the current cell's height. Water can flow from any cell adjacent to an ocean into the ocean.

Return a 2D list of grid coordinates result where result[i] = [ri, ci] denotes that rain water can flow from cell (ri, ci) to both the Pacific and Atlantic oceans.

### Constraints

m == heights.length

n == heights[r].length

1 <= m, n <= 200

0 <= heights[r][c] <= 105

### Initial Thought Process

To start, we need some method that checks each cell only once such that the timing will scale at worst m*n

In principle, this could be very fast by checking around the boundary and ignoring anything that reaches a ridge line.

For example, if there's a ridgeline that is tallest along the diagonal, then it's pointless to check most cells

One initial note, we know that the bottom left and top right corner are guaranteed to already flow into both oceans

For a first pass, I'll check every cell once, and ignore the much more optimal solution of ignoring most cells

### Answer Thought Process

My method essentially updates each value one at a time starting with the lowest points.

If the lowest cells are fully surrounded by taller cells, then I label them with '-'

If they can flow into either ocean, then I label the cells with the respective ocean ('p':Pacific,'a':Atlantic, 'b':both).

I then move onto the next tallest points. With this subselection, they adopt any new value ('p','a','b') of any cell lower than them. This guarantees that if a neighboring cell is lower than the current cell, then it will quickly adopt the neighboring cells value.

For each step, I start by checking if any neighboring cell is lower, and only afterwards if they are equal. This guarantees that there are no race conditions between equal height cells.

This method is nice since it will find ALL cells that can flow to the Pacific only, Atlantic only, neither, or both oceans.

## Gut of Code

In [11]:
import numpy as np

In [64]:
def get_oceans(grid, return_all_cells=False):
    '''
    Finds where the water of each cell ends up
    
    Parameters
    ----------
    grid : 2d integer array of heights with minimum height=0
    return_all_cells : flag indicating whether to return the final location of water from every cell
    
    Returns
    ----------
    both_oceans : list of 2d coordinates desginating which cells lead to both oceans
    grid_water_no_pad : if return_all_cells, grid of final locations of water
    '''
    def _pad(grid):
        '''
        Pads the initial grid with zeros around all edges
        
        Parameters
        ----------
        grid : 2d array of heights of size m by n
        
        Returns
        ----------
        new_grid : 2d array of heights with size m+2 by n+2
        '''
        new_grid = np.zeros((grid.shape[0]+2, grid.shape[1]+2), dtype=int)
        new_grid[1:-1, 1:-1] = grid
        return(new_grid)
    
    def _init_water(shape):
        '''
        Initializes a grid of water where everything in the middle is known 
        and the boundary is known Pacific or Atlantic ocean
        
        Parameters
        ----------
        shape : shape of water grid (should be size of padded grid)
        
        Returns
        ----------
        grid_water : 2d array of values of water
        '''
        grid_water = np.array([['-' for i in range(shape[1])] for j in range(shape[0])])
        grid_water[:1,:] = 'p'
        grid_water[:,:1] = 'p'
        grid_water[-1:,:] = 'a'
        grid_water[:,-1:] = 'a'
        grid_water[-1,0] = 'b'
        grid_water[0,-1] = 'b'
        return(grid_water)
    
    def _combine_water(water, water_sur):
        '''
        Combines the values of the central cell with surrounding cells
        
        Parameters
        ----------
        water : water value of central cell
        water_sur : water values of surrounding cells
        
        Returns
        ----------
        char : a single character of the combined water
        '''
        water_tot = np.insert(water_sur, 0, water)
        if np.any(water_tot=='b'):
            return('b')
        else:
            water_tot = np.unique(water_tot)
            water_tot = water_tot[water_tot!='-']
            if len(water_tot)==2:
                return('b')
            elif len(water_tot)==1:
                return(water_tot[0])
        return('-')
    
    #initialize values
    grid_pad = _pad(grid)
    grid_water = _init_water(grid_pad.shape)
    
    for h in np.sort(np.unique(grid)):
        idx_all = np.argwhere(grid==h)
        check_flat = True
        while(check_flat): #rare case when multiple heights are the same in a row
            check_flat = False
            for idx in idx_all:
                temp_x = np.array([idx[0]+1, idx[0]+2, idx[0]+1, idx[0]])
                temp_y = np.array([idx[1]+2, idx[1]+1, idx[1], idx[1]+1])

                water_sur = grid_water[temp_x, temp_y][h>grid_pad[temp_x, temp_y]]
                water = grid_water[idx[0]+1, idx[1]+1]

                grid_water[idx[0]+1, idx[1]+1] = _combine_water(water, water_sur)
                
                water_sur = grid_water[temp_x, temp_y][h==grid_pad[temp_x, temp_y]]
                water = grid_water[idx[0]+1, idx[1]+1]

                combined_water = _combine_water(water, water_sur)
                grid_water[idx[0]+1, idx[1]+1] = combined_water

                if water!=combined_water:
                    check_flat = True
                    
    grid_water_no_pad = grid_water[1:-1,1:-1]
    
    both_oceans = np.argwhere(grid_water_no_pad=='b')
    
    if return_all_cells:
        return both_oceans, grid_water_no_pad
    else:
        return both_oceans

## Testing Answer

In [67]:
# Simple case to visualize
grid = np.random.randint(low=0, high=5, size=(7,8))
grid

array([[2, 2, 0, 0, 3, 2, 3, 2],
       [1, 0, 4, 4, 3, 3, 2, 0],
       [3, 3, 0, 4, 0, 3, 1, 3],
       [4, 3, 4, 2, 3, 3, 4, 1],
       [3, 3, 1, 4, 0, 1, 3, 0],
       [2, 3, 2, 0, 1, 0, 1, 4],
       [0, 2, 4, 3, 4, 1, 1, 0]])

In [68]:
_, water = get_oceans(grid, return_all_cells=True)
water

array([['p', 'p', 'p', 'p', 'b', 'p', 'b', 'b'],
       ['p', '-', 'b', 'b', 'b', 'b', 'a', 'a'],
       ['b', 'b', '-', 'b', '-', 'b', '-', 'a'],
       ['b', 'b', 'b', '-', 'b', 'b', 'b', 'a'],
       ['b', 'b', '-', '-', '-', '-', 'a', 'a'],
       ['b', 'b', '-', '-', '-', '-', '-', 'a'],
       ['b', 'b', 'b', 'a', 'a', 'a', 'a', 'a']], dtype='<U1')

In [69]:
# Most computationally difficult condition under constraints
grid = np.random.randint(low=0, high=100000, size=(200,200))

In [70]:
get_oceans(grid, return_all_cells=False)

array([[  0, 199],
       [  1, 198],
       [  1, 199],
       [197,   1],
       [197,   2],
       [198,   2],
       [199,   0],
       [199,   1]])

In [71]:
# Non random case with a consistent gradient
grid = np.arange(7)[:,None]+np.arange(8)[None,:]
grid

array([[ 0,  1,  2,  3,  4,  5,  6,  7],
       [ 1,  2,  3,  4,  5,  6,  7,  8],
       [ 2,  3,  4,  5,  6,  7,  8,  9],
       [ 3,  4,  5,  6,  7,  8,  9, 10],
       [ 4,  5,  6,  7,  8,  9, 10, 11],
       [ 5,  6,  7,  8,  9, 10, 11, 12],
       [ 6,  7,  8,  9, 10, 11, 12, 13]])

In [72]:
_, water = get_oceans(grid, return_all_cells=True)
water

array([['p', 'p', 'p', 'p', 'p', 'p', 'p', 'b'],
       ['p', 'p', 'p', 'p', 'p', 'p', 'p', 'b'],
       ['p', 'p', 'p', 'p', 'p', 'p', 'p', 'b'],
       ['p', 'p', 'p', 'p', 'p', 'p', 'p', 'b'],
       ['p', 'p', 'p', 'p', 'p', 'p', 'p', 'b'],
       ['p', 'p', 'p', 'p', 'p', 'p', 'p', 'b'],
       ['b', 'b', 'b', 'b', 'b', 'b', 'b', 'b']], dtype='<U1')